# WEB DATA ANALYTICS PROJECT

### Importing Libraries -

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import csv
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from textblob import TextBlob
import numpy as np

### Initializing lists -

In [34]:
links = []
filter_links = []
final_links = []
final_links2 = []
name = []
quarter = []
year = []
sentiment_final_score = []

### Finding earning calls links - (using selenium)

In [105]:
##### Using selenium to scrap links of Earning Calls #####
## We tried using beautiful soup, however, we were blocked. Hence, using selenium to scrape the web.

for page_num in range(1,3500):
    
    url = 'https://seekingalpha.com/earnings/earnings-call-transcripts/' + str(page_num)
    browser = webdriver.Firefox(executable_path = 'C:/Users/snigb/Desktop/Web Data Analytics/Class 7/geckodriver.exe')
    browser.get(url)
    
    i = 0
    for single_List_Item in browser.find_elements_by_xpath("//ul[@class='list-group sa-base-article-list']/li"):
        i = i+1
        
    for x in range(1,i+1):
        xpath = '//*[@id="analysis-list-container"]/ul/li[' + str(x) + ']/h3/a'
        dropdown = browser.find_element_by_xpath(xpath)
        href_data = dropdown.get_attribute('href')
        links.append(href_data)
    browser.close()

30


### Filtering links for required quarters - 

In [35]:
## Using specific quarters to conduct the analysis and filtering for those quarters

quarters = ["q3-2017","q2-2017","q1-2017","q4-2016","q3-2016","q2-2016","q1-2016","q4-2015","q3-2015",
            "q2-2015","q1-2015","q4-2014","q3-2014","q2-2014","q1-2014","q4-2013","q3-2013","q2-2013",
            "q1-2013","q4-2012","q3-2012","q2-2012","q1-2012"]

filter_links += [link for link in links for quarter in quarters if quarter in link]

### Finding 13-F mutual funds and the respective companies and filtering links for those companies -

For this purpose a separate code has been written called Reading_13F. This code reads all the 13Fs (.txt and .xml) from the sec.gov website and puts them into a dataframe All_Mutual_Funds_Final.csv.

However, for our analysis we are filtering for the companies below as these companies have uniform data present across the years which makes it relevant and apt for analysis.

In [42]:
## Using specific companies to conduct the analysis and filtering for those companies

final_companies = ["alaska-air", "southwest-airlines", "jetblue", "delta-air-lines", "american-airlines","coca-cola-ko",
                   "coca-cola-enterprises", "pepsico", "pfizer","wal-mart", "united-parcel-service", "fedex", "verizon", 
                   "at-and-t", "procter-and-gamble", "johnson-and-johnsons","aapl", "apples-ceo", "comcast", "mmm"]

for link in filter_links:
    for company in final_companies:
        if company in link:
            final_links.append(link)

final_links = list(set(final_links))

for i in range(0,len(final_links)):
    if "results" in final_links[i]:
        final_links2.append(final_links[i])

### Scraping earning calls from seeking alpha - (using beautiful soup)

In [99]:
import urllib2
import bs4 as bs
from bs4 import SoupStrainer
import time
from urllib import FancyURLopener
from random import choice

user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
    'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'
]

In [101]:
##### One methodology of scraping for Earning Calls using Beautiful Soup #####
## We tried using beautiful soup to scrap the earning calls, however, we were blocked. Hence, using selenium.

''''class MyOpener(FancyURLopener, object):
    version = choice(user_agents)

count =0
for link in final_links2:
    myopener = MyOpener()
    page=myopener.open(link)
    html = page.read()

    index = link.find("-")
    link = link[index+1:]
    index = link.find("-")
    name.append(link[:index])
    index = link.find("results")
    quarter.append(link[index-8:index-6])
    year.append(link[index-5:index-1])

    soup = bs.BeautifulSoup(html, 'lxml')

    text = []


    infotable = soup.find_all("div", class_="sa-art article-width")

    for row in infotable:
        a = row.find_all("p", class_="p")
        for i in range(0,len(a)):
            text.append(a[i].getText())''''

##### Using selenium to scrap for Earning Calls #####

browser = webdriver.Firefox(executable_path = 'C:/Users/snigb/Desktop/Web Data Analytics/Class 7/geckodriver.exe')
url = 'https://seekingalpha.com/account/login'
browser.get(url)
USERNAME = browser.find_element_by_xpath("""//*[@id="login_user_email"]""")
USERNAME.send_keys("singh488@purdue.edu")

PASSWORD = browser.find_element_by_xpath("""//*[@id="login_user_password"]""")
PASSWORD.send_keys("chaitchelsea5")

Login_Button=browser.find_element_by_xpath("""//*[@id="orthodox_login"]/div[5]/input""")
Login_Button.click()

for i in range(0, len(final_links2)):
    text = []
    browser.get(final_links2[i])
    dropdown = browser.find_element_by_xpath('//*[@id="main_content"]')
    text.append(dropdown.text)
    
    earning_Call= ''.join(text)
    index = earning_Call.find("All other use is prohibited")

    earning_Call = earning_Call[:index]

##### Using Text Blob for conducting sentiment analysis #####
## This library is used as it is not only simple to understand, but also gives better results as compared to other methods
    
    blob=TextBlob(earning_Call)

    sentiment_score=[]
    for sentence in blob.sentences:
        sentiment_score.append(sentence.sentiment.polarity)

    sentiment_score_df=pd.DataFrame(sentiment_score,columns=['Score'])
    sentiment_final_score.append(sentiment_score_df.Score.mean())

name_sentiment = pd.DataFrame()
name_sentiment = pd.DataFrame({'Name': name, 'Quarter': quarter, 'Year' : year, 'Sentiment_Score' : sentiment_final_score})
name_sentiment.to_csv("Senti_Scores_Final.csv")

### Another methodology of conducting sentiment analysis - (using google cloud)

In [ ]:
##### Demonstrates how to make a simple call to the Natural Language API #####

""""import argparse

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types


def print_result(annotations):
    score = annotations.document_sentiment.score
    magnitude = annotations.document_sentiment.magnitude

    for index, sentence in enumerate(annotations.sentences):
        sentence_sentiment = sentence.sentiment.score
        print('Sentence {} has a sentiment score of {}'.format(
            index, sentence_sentiment))

    print('Overall Sentiment: score of {} with magnitude of {}'.format(
        score, magnitude))
    return 0


def analyze(movie_review_filename):
    client = language.LanguageServiceClient()

    with open(movie_review_filename, 'r') as review_file:
        # Instantiates a plain text document.
        content = review_file.read()

    document = types.Document(
        content=content,
        type=enums.Document.Type.PLAIN_TEXT)
    annotations = client.analyze_sentiment(document=document)

    # Print the results
    print_result(annotations)


if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        description=__doc__,
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument(
        'movie_review_filename',
        help='The filename of the movie review you\'d like to analyze.')
    args = parser.parse_args()

    analyze(args.movie_review_filename)

analyze('earningCall_j_q3.txt')""""

### Aggregating Market Value and Quantity

In [146]:
df = pd.read_csv("All_Mutual_Funds.csv")
df["Date"] = pd.to_datetime(df["Date"])
df["Quarter"] = pd.DatetimeIndex(df['Date']).quarter
df["Year"] = pd.DatetimeIndex(df["Date"]).year
df["Market Value"] = df["Market Value"].str.replace(",", "").astype(int)
df["Quantity"] = df["Quantity"].str.replace(",", "").astype(int)
df["Company"] = df["Company"].str.lower()

In [207]:
main_list=[]
main_list = df.groupby(["Mutual_fund","Company","Year", "Quarter",'Date'], as_index=False)

main_list2 = main_list["Market Value"].mean()
main_list3 = main_list["Quantity"].mean()
main_df = pd.merge(main_list2, main_list3, on=['Mutual_fund',"Company", 'Year', 'Quarter', 'Date'])

main_df['Company'], main_df['Company2'] = main_df['Company'].str.split(' ', 1).str
main_df['Company'], main_df['Company3'] = main_df['Company'].str.split('-', 1).str
main_df['Company'], main_df['Company4'] = main_df['Company'].str.split(',', 1).str
main_df = main_df.drop(['Company2','Company3','Company4'], 1)

In [210]:
main_df.to_csv("All_Mutual_Funds_Final.csv")

### Manipulating S&P 500 stock prices of companies -

In [214]:
sp = pd.read_csv("S&P 500.csv")
sp["Company Name"] = sp["Company Name"].str.lower()
sp['Company'], df['Company2'] = sp['Company Name'].str.split(' ', 1).str
sp = sp.drop(['Company Name','Month'], 1)
#sp.to_csv("S&P500.csv")

### NOTE - 

1. 

2. 

3. 